# Import

#### General Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Model Imports

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.semi_supervised import SelfTrainingClassifier

#### Specific Imports

In [ ]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV


#### General Settings

In [ ]:
train = pd.read_csv('data/stars_train_new.csv', index_col=0)
test = pd.read_csv('data/stars_test_new.csv', index_col=0)

We also initialize the random seed to ensure the reproducibility of our results.

In [ ]:
seed=1

# The Data


### Data Overview

We start by checking if the dataset is clean (duplications, missing values, ...)


In [ ]:
train.shape

In [ ]:
train.duplicated().sum()

In [ ]:
train.isnull().sum()

### Exploratory Data Analysis


#### First Look


In [ ]:
train.head(10)

In [ ]:
train.info()

In [ ]:
train.describe()

All the covariates are continuous. 

#### Look to correlation


We start by examining the relationships between all the variables to discover any patterns.

In [ ]:
sns.pairplot(train)

We observe a strong correlation among the variables $u, g, r, i$ and $z$ as they increase almost linearly, with a larger variation in the higher values, resulting in the observed arrow-like pattern. In this graph, we cannot glean much information about the relationship between the label and the other covariables. Apart from the $\textit{redshift}$ covariable, they appear to be uniformly distributed. To obtain more information, we prefer to create the same graph with the label assigned as the color of the data points, as shown below.

In [ ]:
sns.pairplot(train, hue='label')

In this graph we notice that the points are really $\textit{separable}$ with the $\textit{redshift}$ covariable. 

Now, we represent a heatmap in order to get numerically the correlation already discussed.

In [ ]:
sns.heatmap(train.corr())

As observed in the previous plots, a strong correlation exists among the variables $u, g, r, i,$ and $z$. Such correlations can be problematic for the learning process. Therefore, in the following steps, we will modify the data to retain the information without unnecessary redundancy.

We also need to check if the labels are sufficiently represented in our dataset. 

In [ ]:
sns.countplot(data=train, x='label')

The label 0 is more represented than the others labels. Nevertheless, all the classes are sufficiently represented to be able to learn from them.


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(5, 5))
axes = axes.flatten()
for i in range(len(train.columns[:-1])):
    sns.boxplot(data=train, y=train.columns[i], ax=axes[i])
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(15, 10))
axes = axes.flatten()
for i in range(len(train.columns[:-1])):
    sns.boxplot(data=train, y=train.columns[i], x='label', ax=axes[i])
plt.tight_layout()
plt.show()

We observe for example that as shown in the pairplot that for the $\textit{redshift}$ the first class only takes 0 as a value while the second one is more distributed. Furthermore, as the box of the label 2 is narrower than the others in for the variables $u, g, r, i$ and $z$, it is more concentrated. 

Now, we will try to learn other non-linear structures in the underlying data to discover if it is helpful for the discrimination task. To do this, we introduce the TSNE with multiple perplexity parameters.

In [ ]:
features = train.loc[:, :"redshift"]

tsne = TSNE(n_components=2, random_state=seed)
projections = tsne.fit_transform(features)

In [ ]:
fig = sns.scatterplot(
   x= projections[:,0],y= projections[:,1],
    hue=train["label"]
)
fig

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
perpl=[5, 15, 30,50]
for i in range(len(perpl)):
    tsne = TSNE(n_components=2, random_state=seed, perplexity=perpl[i])
    projections = tsne.fit_transform(features)
    fig = sns.scatterplot(x= projections[:,0],y= projections[:,1], hue=train["label"], ax=axes[i])
    axes[i].set_title(f"Perplexity = {perpl[i]}")
plt.tight_layout()
plt.show()

Unfortunately, we observe that the points are highly mixed between the classes. Therefore, this dimension reduction method may not be helpful for our classification problem.

### Data pre-processing


We begin by splitting the training set into two subsets: a training subset and a testing subset. This allows us to examine the conditional generalization error of our estimate. This initial split is primarily to get an idea of the accuracy, as our main approach for selecting model hyperparameters will involve cross-validation, providing a more accurate estimate of generalization error. 

In [ ]:
X=train.drop("label", axis= 1)
Y=train["label"]
X_train, X_test, Y_train, Y_test=train_test_split(X, Y, stratify= Y, random_state=seed, test_size=0.25)

We standardize the data.

In [ ]:
scaler = StandardScaler()
X_train_sc=scaler.fit_transform(X_train)

As we have noticed from the correlation map, there are numerous highly correlated features. Consequently, having both of them in our analysis might introduce noise rather than valuable information. To mitigate this issue, we will initiate our preprocessing by applying Principal Component Analysis (PCA) 

In [ ]:
pca = PCA(random_state=seed)
train_pca=pca.fit_transform(X_train_sc)

We verify that the total variance is the number of covariates(8).

In [ ]:
pca.explained_variance_.sum()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()
for i in range(4):
    fig = sns.scatterplot(x= train_pca[:,i],y= train_pca[:,i+1], hue=Y_train, ax=axes[i])
    axes[i].set_title(f"Main axes ({i},{i+1})")
plt.tight_layout()
plt.show()

We observe that in these representations the labels are really $\textit{separable}$.

In [ ]:
plt.bar(range(1,9),pca.explained_variance_ratio_)
plt.xlabel("Principal axes")
plt.title("Explained variance ratio")

We observe that the first principal axis primarily explains the variance. However, we will select the number of axes to retain in order to explain at least 95% of the variance ratio.

In [ ]:
print(f"Variance explained with 4 principal axes: {pca.explained_variance_ratio_[:4].sum()}")
print(f"Variance explained with 5 principal axes: {pca.explained_variance_ratio_[:5].sum()}")

Therefore, we keep the first 5 axes.

In [ ]:
X_train_pca=train_pca[:, :5]

We also apply these changes to the test set.

In [ ]:
X_test_pca=pca.transform(scaler.transform(X_test))[:,:5]

### For the submissions

We aim to leverage the entire training dataset for a more comprehensive learning experience, rather than just the portion used for the initial split, with the goal of evaluating our model's F1-score. As a result, we perform PCA and Scaling on the complete training dataset, and subsequently apply the same transformations to the test dataset.

In [ ]:
general_Sc=StandardScaler()
X_sc=general_Sc.fit_transform(X)
general_pca=PCA(random_state=seed)
X_pca=general_pca.fit_transform(X_sc)
X_final=X_pca[:,:5]
test_trans=general_pca.transform(general_Sc.transform(test))[:,:5]

# Learning

We will start with very simple algorithms and continue with more advanced ones. 

## Aggregation

The ultimate goal of this project is to aggregate all the classifiers we have introduced, emphasizing diversity among them. This aligns with the idea behind Random Forests, which constructs base learners with minimal correlation to reduce generalization error. Hence, we will aggregate all the results of our intermediate learners and construct and aggregating method that learns from these results as if they where the initial features. 

In [ ]:
models = []
weight=[]

## General Function

We introduce an easy function that fits the model and gives the f1 score in the test set.

In [ ]:
def learning_testing(model, X_train, Y_train, X_test, Y_test):
    model.fit(X_train, Y_train)
    predictions= model.predict(X_test)
    score = f1_score(Y_test, predictions, average='weighted')
    print(f"Weighted F1 score: \033[91m{score.round(6)}\033[0m for {model.__class__.__name__}")
    return score

In [ ]:
def AggregatePredictions(models, X_train_pca, Y_train, X_test_pca, X_final, Y, test_trans):
    agg_train=pd.DataFrame()
    agg_test=pd.DataFrame()
    for model in models:
        model.fit(X_train_pca, Y_train)
        agg_train[model.__class__.__name__]=model.predict(X_test_pca)
        model.fit(X_final, Y)
        agg_test[model.__class__.__name__]=model.predict(test_trans)
        # Download predictions
        #pd.DataFrame(model.predict(test_trans), index=test.index).to_csv(f"data/predictions_{model.__class__.__name__}.csv")

        return agg_train, agg_test

## Linear Classifier

### Naive Bayes classifier for multivariate Bernoulli models

We start with the simple model of Naive Bayes Classifier for multivariate Bernoulli models.

In [ ]:
bern = BernoulliNB()
weight.append(learning_testing(bern, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(bern)

It does not give a really good accuracy...

### Gaussian Naive Bayes

We try with Gaussian instead of Bernoulli.

In [ ]:
gauss = GaussianNB()
weight.append(learning_testing(gauss, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(gauss)

We have improved a little bit the score.

### Linear Discriminant Analysis

We start with the linear classifiers. To do so, we start by assuming that the distributions of $X$ given the label is gaussian, sharing all the classes the same covariance matrix.

In [ ]:
LDA = LinearDiscriminantAnalysis()
weight.append(learning_testing(LDA, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(LDA)

### Logistic Regression

Now, we are going to use a logistic regression. It is the classifier that minimizes the logistique pseudo-risk.

 We recall that it is regularized. Our penalty will be l2 (l1 penalty is more used for variable selection and in this case it is not the goal as we do not have high dimensional data). 

In [ ]:
lr = LogisticRegression(random_state=seed)
weight.append(learning_testing(lr, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(lr)

Furthermore, we would like to optimize the regulariser parameter. To do so, we apply cross validation. We recall that $C$ is the inverse of the regularization coefficient: larger the value, smaller the regularization. 

In [ ]:
parameters = {'C':[0.5,1,2, 5, 10, 20, 25, 30, 100, 500, 1000]}
lr = LogisticRegression(random_state=seed)
GSlr = GridSearchCV(lr, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSlr.fit(X_final,  Y)
print(f"The best parameters are {GSlr.best_params_} with a score of {GSlr.best_score_} for the model {GSlr.best_estimator_.__class__.__name__}")

In [ ]:
GSlr.cv_results_["rank_test_score"]

We already have $\textit{regularized}$ the model by reducing the number of covariates through PCA, so it is advisable to decrease the regularization, and eventually, we remove the penalty.

In [ ]:
lrNR = LogisticRegression(random_state=seed, penalty=None)
weight.append(learning_testing(lrNR, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(lrNR)

### Perceptron

Finally, we introduce the Perceptron, which is a geometric linear model based on the separability of data.

In [ ]:
parameters = {"penalty":[None, "l2","elasticnet"], "alpha":[0.001, 0.01, 0.1, 1],"max_iter":[500, 750, 1000, 1500, 2000]}
percep = Perceptron(random_state=seed)
GSpercep = GridSearchCV(percep, parameters, scoring="f1_weighted", n_jobs=-1,verbose=3 )
GSpercep.fit(X_final,  Y)
print(f"The best parameters are {GSpercep.best_params_} with a score of {GSpercep.best_score_} for the model {GSpercep.best_estimator_.__class__.__name__}")

In [ ]:
percep = Perceptron(random_state=seed, alpha=0.001, max_iter=500, penalty=None)
weight.append(learning_testing(percep, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(percep)

### Pasive-Aggressive Classifiers

The idea of this family of classifiers it to respond $\textit{passive}$ for the correct answers and $\textit{aggressive}$ for the incorrect ones.

In [ ]:
parameters = {"C":[0.01, 0.1, 0.5, 1, 5],"max_iter":[500, 750, 1000, 1500, 2000]}
psag = PassiveAggressiveClassifier(random_state=seed)
GSpsag = GridSearchCV(psag, parameters, scoring="f1_weighted", n_jobs=-1,verbose=1)
GSpsag.fit(X_final,  Y)
print(f"The best parameters are {GSpsag.best_params_} with a score of {GSpsag.best_score_} for the model {GSpsag.best_estimator_.__class__.__name__}")

In [ ]:
psag = PassiveAggressiveClassifier(random_state=seed, C=0.1, max_iter=500)
weight.append(learning_testing(psag, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(psag)

## No linear classifiers

### Nearest Neighbors

Firstly, we observe that we are in low dimensional dataset thanks to our PCA. Therefore, it makes sense to apply algorithms based on distances. 

In [ ]:
KN=KNeighborsClassifier()
learning_testing(KN, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
parameters = {'n_neighbors':[4, 5, 7, 10, 13, 15, 20, 30, 50, 75]}
KN = KNeighborsClassifier()
GSKN = GridSearchCV(KN, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSKN.fit(X_final,  Y)
print(f"The best parameters are {GSKN.best_params_} with a score of {GSKN.best_score_} for the model {GSKN.best_estimator_.__class__.__name__}")

In [ ]:
kn = KNeighborsClassifier(n_neighbors=5)
weight.append(learning_testing(kn, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(kn)

 ### QDA

Instead of assuming the same covariance for all covariate distributions conditioned on the labels, we assume that each one has a different covariance matrix. This makes the decision frontier quadratic, in contrast to the linear decision boundary in LDA.

In [ ]:
QDA=QuadraticDiscriminantAnalysis()
weight.append(learning_testing(QDA, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(QDA)

### SVM

The Support Vector Machine (SVM), a method that aims to construct a discriminating hyperplane that maximizes the margins between classes. Once this it is computed, the predictions will be done by taking the sign of the signed distance to the hyperplane. This hyperplane is computed by minimizing the hinge loss, which is given by $$\Phi(X):=(1-X)_{+},$$
where $(x)_+:=\mathrm{max}(0,x)$. Furthermore, there is also a term that can be interpreted as a Ridge regularisation. Therefore, the SVM can be seen as the following minimisation problem:
\begin{align*}
    \mathrm{min}_{w,b}\frac{1}{2}\|w \|^2+C\sum_{i=1}^n\left(1-Y_i(w^\top X_i+b)\right)_+,
\end{align*}
so that we are penalizing the signed distance to the hyperplane defined by $w$ and $b$. 
The regularization parameter $C$, works inversely to the regularization strength. A higher value of $C$ leads to increased penalization of misclassified observations or those near the margin. Consequently, this results in a more complex construction of the margin which is less $\textit{regularized}$.

At the end, only the points that are too close to the margin or those that are misclassified will have an impact on constructing the separating hyperplane.

In [ ]:
SVClin=LinearSVC(random_state=seed, max_iter=2500)
learning_testing(SVClin, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
parameters = { "tol":[1e-4,1e-5, 1e-6], "C":[0.5,1,5,10, 50, 100], "max_iter":[750,1000,1500, 2000]}
SVClin = LinearSVC(random_state=seed, loss="squared_hinge", multi_class="crammer_singer")
GSSVClin= GridSearchCV(SVClin, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSSVClin.fit(X_final,  Y)
print(f"The best parameters are {GSSVClin.best_params_} with a score of {GSSVClin.best_score_} for the model {GSSVClin.best_estimator_.__class__.__name__}")

In [ ]:
SVClin=LinearSVC(random_state=seed, C=50, max_iter=750, tol=0.0001, loss="squared_hinge", multi_class="crammer_singer")
weight.append(learning_testing(SVClin, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(SVClin)

### Polynomial SVM

In this section, we try the polynomial kernel, given by $$K_{\mathrm{poly}}(x,y):=\left(\gamma\langle x,y\rangle+c\right)^d,$$
 where $d$ represents the degree and $c$ and $\gamma$ are coefficients controlling the significance of higher-order versus lower-order polynomials. In the Scikit-learn library, the value of the coefficient $c$ can be adjusted using the parameter `coef0`, the value of $d$ can be modified using the parameter `degree`, and the value of $\gamma$ by `gamma`.

In [ ]:
SVCpol=SVC(random_state=seed, kernel="poly", degree=3, C=100)
learning_testing(SVCpol, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
#parameters = {"degree":[3, 4], "C":[1,10, 50, 100]}
parameters = {"degree":[3], "C":[100, 150, 200, 300]}
SVCpol = SVC(tol=0.001, kernel="poly", random_state=seed)
GSSVCpol = GridSearchCV(SVCpol, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSSVCpol.fit(X_final,  Y)
print(f"The best parameters are {GSSVCpol.best_params_} with a score of {GSSVCpol.best_score_} for the model {GSSVCpol.best_estimator_.__class__.__name__}")

In [ ]:
SVCpol=SVC(tol=0.001, kernel="poly", random_state=seed, degree=3, C=200)
weight.append(learning_testing(SVCpol, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(SVCpol)

### Other kernels SVM

We have also tried other different kernels such as the Gaussian one, which is usually referred as $\textit{Radial Basis Function}$(RBF) and which is computed by
$$K_{\mathrm{RBF}}(x,y):=\mathrm{exp}\left(-\gamma\|x-y\|^2\right),$$
where the $\gamma$ can be seen as the inverse of the $2\sigma^2$ of a Gaussian distribution, so as the $\gamma$ increases, the variance decreases, causing the kernel to yield smaller values. This parameter can be tuned by the parameter `gamma` in Scikit-learn.

 Finally, another kernel denoted as Sigmoid is given by
 $$K_{\mathrm{sigm}}(x,y):=\mathrm{tanh}\left(\langle x , y\rangle +r\right),$$
where $\alpha$ represents the parameter `alpha` and $r$ the parameter `coef0`.

In [ ]:
SVCkern=SVC(C= 1000, decision_function_shape= 'ovo', gamma= 'auto', kernel='rbf')
learning_testing(SVCkern, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
#parameters = {"kernel":["rbf", "sigmoid"], "gamma":["scale", "auto"], "C":[3,5,10],"decision_function_shape":["ovo", "ovr"]}
parameters = { "C":[10, 50, 100, 500, 1000]}
SVCkern = SVC(random_state=seed, decision_function_shape="ovo", gamma="auto", kernel="rbf")
GSSVCKern = GridSearchCV(SVCkern, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSSVCKern.fit(X_final,  Y)
print(f"The best parameters are {GSSVCKern.best_params_} with a score of {GSSVCKern.best_score_} for the model {GSSVCKern.best_estimator_.__class__.__name__}")

In [ ]:
SVCkern=SVC(random_state=seed, decision_function_shape="ovo", gamma="auto", kernel="rbf", C=500)
weight.append(learning_testing(SVCkern, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(SVCkern)

### Decision Tree

This method attempts to partition the space by posing binary questions, each on a single variable. We end up separating the covariate space $\mathcal{X}$ in $m$ regions given by $(R_1,... ,R_m)$. Then, the classifier is given by
$$ f(x) = \sum ^m_{j=1} v_j \mathbb{1}_{x\in R_j}. $$

The $v_j$ is going to be the most voted class of the leaf.

It is highly flexible but faces the inconvenience of potentially interpolating the entire dataset. To manage this, we tested multiple parameters, like the minimum number of samples required to split a leaf or the maximum allowed depth.

In [ ]:
tree=DecisionTreeClassifier(random_state=seed)
learning_testing(tree, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
parameters = {"criterion":["gini", "entropy", "log_loss"], "max_depth":[5, 10, 12, 15, 20, 30], "min_samples_split":[2, 5,10, 20], "min_samples_leaf":[1, 2, 5, 10]}
tree = DecisionTreeClassifier(random_state=seed)
GSTree = GridSearchCV(tree, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSTree.fit(X_final,  Y)
print(f"The best parameters are {GSTree.best_params_} with a score of {GSTree.best_score_} for the model {GSTree.best_estimator_.__class__.__name__}")

In [ ]:
tree = DecisionTreeClassifier(random_state=seed, criterion="entropy", max_depth=30, min_samples_leaf=10, min_samples_split=2)
weight.append(learning_testing(tree, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(tree)

### Random Forest

It aims to aggregate multiple decision trees as uncorrelated as possible. To do so,it leverages the bootstrap technique, involving resampling the data with replacement to simulate new datasets. A decision tree is trained for each bootstrap sample. The final predictor is determined by averaging the individual predictors:
$$ \hat f _{bag}(\mathcal D_n)(x) = \frac{1}{n} \sum^b _{i=1}\hat f_i(x). $$
The primary advantage of this method is its reduction of variance while sustaining an equal level of bias. 

 we also seek to enhance the method by fine-tuning multiple parameters, such as the number of estimators or the hyperparameters inherited by the decision tree, like the maximum depth.

In [ ]:
RF=RandomForestClassifier(random_state=seed, n_estimators=200, max_depth=20)
learning_testing(RF, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
parameters = {"n_estimators":[100,150, 250, 500], "max_depth":[10, 15, 20, 30, 50]}#, "min_samples_split":[2, 5,10, 20], "min_samples_leaf":[1, 2, 5, 10],
RF = RandomForestClassifier(random_state=seed, max_features=3)
GSRF = GridSearchCV(RF, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSRF.fit(X_final,  Y)
print(f"The best parameters are {GSRF.best_params_} with a score of {GSRF.best_score_} for the model {GSRF.best_estimator_.__class__.__name__}")

In [ ]:
RF = RandomForestClassifier(random_state=seed, max_features=3, max_depth=50, n_estimators=150)
weight.append(learning_testing(RF, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(RF)

### Neural Network: Multilayer Perceptron Classifier

A neural network consists of a series of interconnected layers, each formed by multiple adapted weight combinations from the preceding layer. These combinations are activated by a function to introduce non-linearity, a key aspect that allows neural networks to act as universal approximators.

Despite the opacity of these models, their consistent accuracy across numerous cases makes us consider them in our models.

In this project, we focus on using the methods already implemented in the $\textit{MLPClassifier}$ model of Scikit-learn. This includes optimizing essential parameters like the number and size of layers, choice of optimizers, activation functions, the number of iterations, and strategies for learning rate optimization.

In [ ]:
mlp=MLPClassifier(random_state=seed, max_iter=500, hidden_layer_sizes=[100,100,100])
learning_testing(mlp, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
#parameters = { "max_iter":[400, 500, 600, 700], "alpha":[1e-3, 1e-4, 1e-5]}#"max_iter":[200, 500, 1000], "learning_rate":["constant", "invscaling","adaptive"], "alpha":[1e-3, 1e-4, 1e-5],
parameters = { "max_iter":[400, 600, 1000], "alpha":[1e-5],"activation":["relu","logistic"], "hidden_layer_sizes":[[50,50, 40, 60], [100, 30, 30, 20, 20, 20,10]]}
mlp = MLPClassifier(random_state=seed, solver="lbfgs", learning_rate="constant")
GSmlp = GridSearchCV(mlp, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSmlp.fit(X_final,  Y)
print(f"The best parameters are {GSmlp.best_params_} with a score of {GSmlp.best_score_} for the model {GSmlp.best_estimator_.__class__.__name__}")

In [ ]:
mlp = MLPClassifier(random_state=seed, solver="lbfgs", activation="logistic", learning_rate="constant", alpha=1e-05, max_iter=600)
weight.append(learning_testing(mlp, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(mlp)

### Boosting

It is an aggregation method, different from bagging, where the base predictors are not built in parallel and independently. Instead, they are constructed iteratively based on the residuals from previous iterations. 
The initial problem is given by $$\widehat{f}\in \underset{(h_m, \alpha_m)_{1\leq m\leq M}}{\mathrm{argmin}}{\frac{1}{n}\sum_{i=1}^nc\left(\sum_{m=1}^M\alpha_mh_m(X_i), Y_i\right)}.$$
However, this approach is not feasible. Then, as detailed in the report for the regression boosting, a simplification is proposed. In fact, with this simplification it is possible to compute an explicit iterative model for the exponential loss: Adaboost. 
However, we observe better score with log loss.

In [ ]:
gradb=GradientBoostingClassifier(random_state=seed, n_estimators=300, max_depth=10, learning_rate=1)
learning_testing(gradb, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
parameters = { "n_estimators":[100, 200, 300], "learning_rate":[0.1,0.5, 1], "max_depth":[5, 10, 15]}#"criterion":["friedman_mse", "squared_error"],
gradb = GradientBoostingClassifier(random_state=seed, loss="log_loss", max_features=3)
GSgradb = GridSearchCV(gradb, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSgradb.fit(X_final,  Y)
print(f"The best parameters are {GSgradb.best_params_} with a score of {GSgradb.best_score_} for the model {GSgradb.best_estimator_.__class__.__name__}")

In [ ]:
gradb = GradientBoostingClassifier(random_state=seed, loss="log_loss", max_features=3, learning_rate=0.5, max_depth=5, n_estimators=300)
weight.append(learning_testing(gradb, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(gradb)

Now, we are trying with XGboost and Catboost

In [ ]:
xgb=XGBClassifier()
learning_testing(xgb, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
params={'n_estimators': [100, 200], 'max_depth': range(2,25)}
xgb = XGBClassifier()
GSxgb = GridSearchCV(xgb, params, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSxgb.fit(X_final,  Y)
print(f"The best parameters are {GSxgb.best_params_} with a score of {GSxgb.best_score_} for the model {GSxgb.best_estimator_.__class__.__name__}")

In [ ]:
xgb = GradientBoostingClassifier(random_state=seed, loss="log_loss", n_estimators=200, max_depth=6, max_features='sqrt', learning_rate=0.1)
weight.append(learning_testing(xgb, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(xgb)

In [ ]:
cat = CatBoostClassifier()
learning_testing(cat, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
params={'iterations':[300, 600], 'learning_rate':[0.05, 0.1,0.2], 'bootstrap_type':['Bayesian', 'Bernoulli']}
cat = CatBoostClassifier()
GScat = GridSearchCV(cat, params, scoring="f1_weighted", n_jobs=-1, verbose=3)
GScat.fit(X_final,  Y)
print(f"The best parameters are {GScat.best_params_} with a score of {GScat.best_score_} for the model {GScat.best_estimator_.__class__.__name__}")

In [ ]:
cat = CatBoostClassifier(bootstrap_type='Bayesian', iterations=600, learning_rate=0.2)
weight.append(learning_testing(cat, X_train_pca, Y_train, X_test_pca, Y_test))
models.append(cat)

## Aggregation

In [ ]:
agg_train, agg_test = AggregatePredictions(models, X_train_pca, Y_train, X_test_pca, X_final, Y, test_trans)

We start by selecting the label that received the most votes among the classifiers

In [ ]:
avg_agg=agg_test.mode(axis=1).iloc[:,0]

In [ ]:
#pd.DataFrame(np.array(avg_agg), index=test.index).to_csv('data/predictions_avg.csv')

Now, we will calculate a $\textit{weighted}$ mode to assign varying importance to different classifiers, taking into account their varying levels of accuracy. Our approach involves assigning each classifier a weight based on their F1-score, with the best classifiers receiving higher weights.

In [ ]:
def weigth_mode(x):
    accum=np.zeros(3)
    for i in range(len(x)):
        accum[x[i]]+=weight[i]
    return np.argmax(accum)

In [ ]:
weight_agg=agg_test.apply(weigth_mode, axis=1)

In [ ]:
print(f"Weighted F1 score: \033[91m{f1_score(Y_test, weight_agg, average='weighted').round(6)}\033[0m")
#pd.DataFrame(np.array(weight_agg), index=test.index).to_csv('weight.csv')

Following the same idea, we will train another method to better aggregate the base learners

In [ ]:
parameters = {"n_estimators":[100,200, 300, 350], "max_depth":[10, 20],"min_samples_split":[2, 5,10, 20], "min_samples_leaf":[1, 2, 5, 10] }
RF = RandomForestClassifier(random_state=seed)
GSRFag = GridSearchCV(RF, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSRFag.fit(agg_train,  Y_train)
print(f"The best parameters are {GSRFag.best_params_} with a score of {GSRFag.best_score_} for the model {GSRFag.best_estimator_.__class__.__name__}")

In [ ]:
rfAgpred=GSRFag.best_estimator_.predict(agg_test)
print(f"Weighted F1 score: \033[91m{f1_score(Y_test, rfAgpred, average='weighted').round(6)}\033[0m")
#pd.DataFrame(rfAgpred, index=test.index).to_csv('rfAgpred.csv')

In [ ]:
parameters = { "max_iter":[300, 500, 600, 800], "alpha":[1e-5]}#"max_iter":[200, 500, 1000], "learning_rate":["constant", "invscaling","adaptive"], "alpha":[1e-3, 1e-4, 1e-5],
mlp = MLPClassifier(random_state=seed, solver="lbfgs", activation="logistic", learning_rate="constant")
GSmlp = GridSearchCV(mlp, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSmlp.fit(agg_train,  Y_train)
print(f"The best parameters are {GSmlp.best_params_} with a score of {GSmlp.best_score_} for the model {GSmlp.best_estimator_.__class__.__name__}")

In [ ]:
mlpAgpred=GSmlp.best_estimator_.predict(agg_test)
print(f"Weighted F1 score: \033[91m{f1_score(Y_test, mlpAgpred, average='weighted').round(6)}\033[0m")
#pd.DataFrame(mlpAgpred, index=test.index).to_csv('data/predictions_mlpAgpred.csv')

First, we prepare the data. Since our objective is diversity, we will introduce the initial data even before applying PCA to ensure that the $\textit{base learners}$ are as uncorrelated as possible. Therefore, we achieve this uncorrelation by constructing the classifiers using different covariates.

In [ ]:
X_ag=pd.concat([X_test, agg_train], axis=1, ignore_index=True)

In [ ]:
X_ag = pd.DataFrame(index=range(X_test.shape[0]), columns=range(X_test.shape[1]+agg_train.shape[1]))
X_ag.iloc[:, :X_test.shape[1]] = X_test.values
X_ag.iloc[:, X_test.shape[1]:] = agg_train.values

In [ ]:
test_ag = pd.DataFrame(index=range(test.shape[0]), columns=range(test.shape[1]+agg_test.shape[1]))
test_ag.iloc[:, :test.shape[1]] = test.values
test_ag.iloc[:, test.shape[1]:] = agg_test.values

In [ ]:
parameters = {"n_estimators":[100,200,250, 275, 300, 350], "max_depth":[10, 20, 25, 30, 40],"min_samples_split":[2, 5,10], "min_samples_leaf":[1,  5, 10], "max_features":[3,4,5,6,8] }
RF2 = RandomForestClassifier(random_state=seed)
GSRFag2 = GridSearchCV(RF2, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSRFag2.fit(X_ag,  Y_test)
print(f"The best parameters are {GSRFag2.best_params_} with a score of {GSRFag2.best_score_} for the model {GSRFag2.best_estimator_.__class__.__name__}")

In [ ]:
RFAgpred2=GSRFag2.best_estimator_.predict(test_ag)
#pd.DataFrame(RFAgpred2, index=test.index).to_csv('data/predictions_RFAgpred2.csv')

In [ ]:
parameters = { "max_iter":[300,400, 500, 600, 700, 900],"alpha":[1e-3, 1e-4, 1e-5], "activation": ["logistic", "relu"],"learning_rate":["constant", "invscaling","adaptive"]}#"max_iter":[200, 500, 1000], "learning_rate":["constant", "invscaling","adaptive"], "alpha":[1e-3, 1e-4, 1e-5],
mlp2 = MLPClassifier(random_state=seed, solver="lbfgs")
GSmlp2 = GridSearchCV(mlp2, parameters, scoring="f1_weighted", n_jobs=-1, verbose=3)
GSmlp2.fit(X_ag,  Y_test)
print(f"The best parameters are {GSmlp2.best_params_} with a score of {GSmlp2.best_score_} for the model {GSmlp2.best_estimator_.__class__.__name__}")

In [ ]:
mlpAgpred2=GSmlp2.best_estimator_.predict(test_ag)
#pd.DataFrame(mlpAgpred2, index=test.index).to_csv('data/predictions_mlpAgpred2.csv')

## Semi-supervised methods

We adopt a transductive perspective, treating the training set as our labeled data $\mathcal{D}_n$ and the test set as the unlabeled data $\mathcal{U}_m$. This approach assumes that the entire set we aim to predict is available from the outset.

### Self-training

We start with the already implemented heuristic of self-training that iteratively adds to the labeled data the unlabeled data for which we have a confident prediction.

In [ ]:
#svc = SVC(probability=True, gamma="auto")
log=LogisticRegression(random_state=seed)
self_training_model = SelfTrainingClassifier(log)
X_semi=X.append(test)
Y_semi=Y.append( pd.Series([-1] * test.shape[0]))
stLog=self_training_model.fit(X_semi, Y_semi)

In [ ]:
stLogR=stLog.predict(test)
#pd.DataFrame(stLogR, index=test.index).to_csv('data/predictions_stLogR.csv')

# Robust Self Training

Now, we are going to build a robust version of the previous algorithm to add only the unlabeled data for which all the predictors where confident.

In [ ]:
class RobustSelfTrainingClassifior():
    def __init__(self, Models):
        self.models=Models.copy()

    def fit(self, X, y, max_iter, seed=seed):
        X_labeled, X_unlabeled, y_labeled, y_unlabeled=train_test_split(X, y, stratify=y, random_state=seed, test_size=0.25)
        print(f'Initial labeled data: {len(X_labeled)}, Initial unlabeled data: {len(X_unlabeled)}')

        for i in range(max_iter):
            # Train models
            for name, model in self.models.items():
                model['model'].fit(X_labeled, y_labeled)
            
            # Predict unlabeled data
            Y_pred_dict = {}
            for name, model in self.models.items():
                pred = model['model'].predict_proba(X_unlabeled)
                pred = pd.DataFrame(pred, index=X_unlabeled.index)
                Y_pred_dict[name] = {'model_pred' : pred, 'tau': model['tau']}

            # Create each set for each model
            S = []
            for name, model in Y_pred_dict.items():
                model_pred = model['model_pred']
                tau = model['tau']
                S.append([set(model_pred.index[model_pred.iloc[:, i] > tau].tolist()) for i in range(int(y_labeled.min()), int(y_labeled.max())+1)])
            # Get only the index of the unlabeled data which predict by all models
            S = np.transpose(S)
            S = [set.intersection(*item) for item in S]
            S = set.union(*S)

            # break the for if S is empty
            if not S:
                print('No unlabelled data unanimously predicted')
                break

            # Add the new labeled data to the labeled data
            X_labeled = pd.concat([X_labeled, X_unlabeled.loc[list(S)]], axis=0)
            # Delete the new labeled data from the unlabeled data
            X_unlabeled = X_unlabeled.drop(list(S), axis=0)
            # Add the new labeled data to the labeled data
            y_labeled = pd.concat([y_labeled, y_unlabeled.loc[list(S)]], axis=0)
            if X_unlabeled.shape[0] == 0:
                print('All unlabeled data are labeled')

            print(f'Iteration {i+1} finished, {len(X_labeled)} labeled data, {len(X_unlabeled)} unlabeled data')

        else:
            print('Max iteration reached')
        
        return self
    
    def predict(self, X):
        # List of matrix prediction of each model
        Y_list = []
        for name, model in self.models.items():
            Y_list.append(pd.DataFrame(model['model'].predict_proba(X), index=X.index))
        # Average every prediction probability
        Y = pd.concat(Y_list).groupby(level=0).mean()
        # Get the index of the max probability
        Y['target'] = Y.idxmax(axis=1)
        # Get only the target column
        Y = Y[['target']]

        return Y

In [ ]:
def FindTau(model,  X_train, Y_train, X_val, Y_val):
    """
    Find the best tau which maximize the f1 score

    Parameters:
    -----------
    model: model to find the best tau
    X_train: train data
    Y_train: train labels
    X_val: validation data
    Y_val: validation labels

    Returns:
    --------
    best_tau: best tau which maximize the f1 score
    """
    model.fit(X_train, Y_train)
    pred = model.predict_proba(X_val)
    pred = np.append(pred, np.zeros((pred.shape[0], 1)), axis=1).copy()
    best_tau = 0
    best_accuracy = 0
    for tau in np.arange(0.5,1,0.025):
        Y_pred = (pred >= tau).astype(int)
        Y_pred[np.sum(Y_pred, axis=1) == 0, 3] = 1
        Y_pred = np.argmax(Y_pred, axis=1)
        accuracy = f1_score(Y_val, Y_pred, average='weighted')
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_tau = tau
        #print(f"tau: {tau:.2f}, accuracy: {accuracy:.3f}")
        
    print(f"Best tau for the model {model.__class__.__name__} is {best_tau}")
    return best_tau

In [ ]:
tau_logistic = FindTau(lrNR, X_train_pca, Y_train, X_test_pca, Y_test)
tau_cat = FindTau(cat, X_train_pca, Y_train, X_test_pca, Y_test)
tau_RF = FindTau(RF, X_train_pca, Y_train, X_test_pca, Y_test)

In [ ]:
models = {
    'logistic': {'model': lrNR, 'tau': tau_logistic},
    'cat': {'model': cat, 'tau': tau_cat},
    'RF': {'model': RF, 'tau': tau_RF},
}

In [ ]:
RSTC = RobustSelfTrainingClassifior(models).fit(pd.DataFrame(X_train_pca, index=Y_train.index), Y_train, max_iter=20)

In [ ]:
print(f"Weighted F1 score: \033[91m{f1_score(Y_test, RSTC.predict(pd.DataFrame(X_test_pca, index=Y_test.index)), average='weighted').round(6)}\033[0m")

The Robust Self-Training is not accurate...

In [ ]:
RSTC = RobustSelfTrainingClassifior(models).fit(pd.DataFrame(X_final, index=Y.index), pd.DataFrame(Y), 50)
RSTCpred=RSTC.predict(pd.DataFrame(test_trans))
#pd.DataFrame(RSTCpred, index=test.index).to_csv('data/predictions_RSTCpred.csv')